In [2]:
import pandas as pd

In [13]:
df = pd.read_csv("train.csv", index_col = 0)
df.index = df['id']
df.drop(columns = "id")

print(df.head())
print(df.tail())


              id                                        arch_and_hp  \
id                                                                    
train_0  train_0  Sequential( (conv0): Conv2d(3, 28, kernel_size...   
train_1  train_1  Sequential( (batchnorm0): BatchNorm2d(3, eps=1...   
train_2  train_2  Sequential( (batchnorm0): BatchNorm2d(3, eps=1...   
train_3  train_3  Sequential( (conv0): Conv2d(3, 32, kernel_size...   
train_4  train_4  Sequential( (batchnorm0): BatchNorm2d(3, eps=1...   

         batch_size_test  batch_size_val           criterion  epochs  \
id                                                                     
train_0             1024             512  CrossEntropyLoss()     590   
train_1             1024             512  CrossEntropyLoss()     556   
train_2             1024             512  CrossEntropyLoss()     556   
train_3             1024             512  CrossEntropyLoss()     550   
train_4             1024             512  CrossEntropyLoss()     404  